In [ ]:
#uses Registered#Nto1Adder_NIR

In [ ]:
#registerings implemented based on the assumption that delay of all Registered#Nto1Adder_NIR is 1 clk

In [ ]:
import sys
import math
import io

In [ ]:
import import_ipynb
from RegisteredNto1Adder_V1_1 import RegisteredNto1Adder

In [ ]:
def AdderTree_ASm(parameters, Print_To_File, Files_Location, Is_Top, modules):
    
    N = parameters["N"]
    AdderSize = parameters["ADDER_SIZE"]
    
    #IN_WIDTH>=1
    IN_WIDTH = 10

    ModuleName="AdderTree_{}_A{}".format(N, AdderSize)

    if Is_Top:
        Files_Location += ModuleName+"/"
        if not os.path.exists(Files_Location):
            os.makedirs(Files_Location)

    if Print_To_File<=0:
        of=sys.stdout
    else:
        of=open(Files_Location+ModuleName+".v", 'w+')

    AdderSizes = set()

    print("`timescale 1ns / 1ps\n", file=of)
    print("module "+ModuleName, file=of)
    print("#(", file=of)
    print("parameter IN_WIDTH = {}".format(IN_WIDTH), file=of)
    print(")(", file=of)
    print("input clk, reset, enable,", file=of)
    print("input inReady,", file=of)
    print("input signed [IN_WIDTH-1:0] ", end='', file=of)
    for i in range(N):
        print("A{}, ".format(i), end='', file=of)
    print(file=of)
    print("output outReady,", file=of)
    lgn=math.ceil(math.log(N,2))
    al=-1+lgn
    if al==0:
        print("output signed [IN_WIDTH:0] out,", file=of)
    else:
        print("output signed [IN_WIDTH+{}:0] out,".format(al), file=of)
    print("output earlyOutReady", file=of)
    print(");\n", file=of)

    regsBuf=io.StringIO()
    AIBuf=io.StringIO()

    maxLevel=math.ceil(math.log(N,AdderSize))
    #print("maxLevel = {}".format(maxLevel))
    level=0

    nextElememtNo=N #remained
    sizeList=[]
    for i in range(N):
        sizeList.append(1)

    i=0
    new = N

    REGimp=False

    while level!=maxLevel:
        #print("StartNextElememtNo = {}".format(nextElememtNo))
        currentElememtNo = nextElememtNo
        nextElememtNo=0
        ORimp=False
        while currentElememtNo>0:
            if currentElememtNo >= AdderSize:
                CurrentAdderSize = AdderSize
                currentElememtNo -= AdderSize
            else:
                CurrentAdderSize = currentElememtNo
                currentElememtNo = 0
            nextElememtNo=nextElememtNo+1
            #print("nextElememtNo = {}".format(nextElememtNo))
            outSize=0
            for k in range (i, i+CurrentAdderSize):
                outSize+=sizeList[k]
            sizeList.append(outSize)
            if CurrentAdderSize==1: #just reg
                if outSize==1:
                    AIBuf.write("reg signed [IN_WIDTH-1:0] M{};\n".format(new))
                    #print file=of
                elif outSize==2:
                    AIBuf.write("reg signed [IN_WIDTH:0] M{};\n".format(new))
                    #print file=of
                else:
                    AIBuf.write("reg signed [IN_WIDTH+{}:0] M{};\n".format(math.ceil(math.log(outSize,2))-1, new))
                    #print file=of
                if level==0:
                    regsBuf.write("\t\tif(inReady) begin\n")
                    regsBuf.write("\t\t\tM{} <= A{};\n".format(new, i))
                else:
                    regsBuf.write("\t\tif(OR[{}]) begin\n".format(level-1))
                    regsBuf.write("\t\t\tM{} <= M{};\n".format(new, i))      
                regsBuf.write("\t\tend\n")
                i+=1
                REGimp=True
            else: #Generate Adder
                if level!=maxLevel-1:
                    if outSize==2: #outSize>=2
                        AIBuf.write("wire signed [IN_WIDTH:0] M{};\n".format(new))
                    else:
                        AIBuf.write("wire signed [IN_WIDTH+{}:0] M{};\n".format(math.ceil(math.log(outSize,2))-1, new))
                inSizeL = math.ceil(math.log(sizeList[i],2))
                #print("inSize = {}".format(inSize))
                rE = i+CurrentAdderSize
                eSizeList=[]
                for k in range (i, rE):
                    curSizeL=math.ceil(math.log(sizeList[k],2))
                    eSizeL=inSizeL-curSizeL
                    eSizeList.append(eSizeL)
                    #print("esize = {}".format(eSizeL))
                    if eSizeL!=0:
                        if inSizeL==1:
                             AIBuf.write("wire signed [IN_WIDTH:0] M{}e=".format(k))
                        else:
                             AIBuf.write("wire signed [IN_WIDTH+{}:0] M{}e=".format(inSizeL-1, k))
                        AIBuf.write("{{")
                        AIBuf.write("{}".format(eSizeL))
                        AIBuf.write("{")
                        AIBuf.write("M{}[IN_WIDTH".format(k))
                        if curSizeL==0:
                            AIBuf.write("-1")
                        elif curSizeL>1:
                            AIBuf.write("+{}".format(curSizeL-1))
                        AIBuf.write("]}},")
                        AIBuf.write("M{}".format(k))
                        AIBuf.write("};\n")
                AIBuf.write("Registered{}to1Adder_NIR #(.IN_WIDTH(IN_WIDTH".format(CurrentAdderSize))
                
                AdderSizes.add(CurrentAdderSize)
                
                curSizeL=math.ceil(math.log(sizeList[i],2))
                if inSizeL>0:
                    AIBuf.write("+{}".format(inSizeL))
                AIBuf.write("))\nAdd{}(\nclk, reset, enable, \n".format(new))
                if level==0:
                    AIBuf.write("inReady,\n")
                    for k in range (i, rE):
                        AIBuf.write("A")
                        AIBuf.write("{}, ".format(k))
                else:
                    AIBuf.write("OR[{}],\n".format(level-1))
                    for k in range (i, rE):
                        AIBuf.write("M")
                        AIBuf.write("{}".format(k))
                        if(eSizeList[k-i]!=0):
                            AIBuf.write("e");
                        AIBuf.write(", ")
                AIBuf.write("\n")
                if level==maxLevel-1:
                    AIBuf.write("outReady,\n")
                else:
                    if ORimp==False:
                        AIBuf.write("OR[{}],\n".format(level))
                        ORimp=True
                    else:
                        AIBuf.write("ORa{}, //not used\n".format(new))
                if level==maxLevel-1:
                    AIBuf.write("out,\n")
                else:
                    AIBuf.write("M{},\n".format(new))
                if level==maxLevel-1:
                    AIBuf.write("earlyOutReady);\n\n")        
                else :
                    AIBuf.write("aeor{}); //not used\n\n".format(new))
                i+=CurrentAdderSize
            new+=1
        level+=1

    if maxLevel>1:
        print("wire [0:{}] OR;\n".format(maxLevel-2), file=of)

    print(AIBuf.getvalue(), end='',file=of)

    if REGimp:
        print("always @(posedge clk) begin", file=of)
        print("\tif(reset) begin", file=of)
        print("\t\t//do nothing", file=of)
        print("\tend", file=of)
        print("\telse if(enable) begin", file=of)
        print(regsBuf.getvalue(), end='',file=of)
        print("\tend", file=of)
        print("end\n", file=of)

    print("endmodule", file=of)

    if Print_To_File>0:
        of.close()
    
    temp_e_EIR = False
    if "ENABLE_INPUT_REGISTERS" in parameters:
        temp_e_EIR = True
        temp_EIR = parameters["ENABLE_INPUT_REGISTERS"]
    parameters["ENABLE_INPUT_REGISTERS"] = 0
    for AS in AdderSizes:
        parameters["N"] = AS 
        RegisteredNto1Adder(parameters, Print_To_File, Files_Location, 0, modules)
    if temp_e_EIR:
        parameters["ENABLE_INPUT_REGISTERS"] = temp_EIR
    parameters["N"] = N
    
    modules[ModuleName]="AdderTree_ASm"
    return ModuleName